In [1]:
import pyspark

In [2]:
conf = pyspark.SparkConf()
conf.setMaster('spark://spark-master:7077')
spark_context = pyspark.SparkContext(conf=conf)

In [3]:
data = spark_context.textFile('/user-data/combined/reviews.csv')

In [ ]:
header_data = data.first()

In [ ]:
 def calculate_weight(helpful_yes, helpful_no):
    return helpful_yes / (helpful_yes + helpful_no)

In [ ]:
def helpful_data_is_digits(value1, value2):
    try:
        float(value1)
        float(value2)
        return True
    except ValueError:
        return False

In [ ]:
def heplful_data_is_valid(value1, value2):
    if not helpful_data_is_digits(value1, value2):
        return False
    return float(value1) > 0 and float(value2) > 0

In [ ]:
result_data = data.filter(lambda row: row != header_data) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line) > 5) \
    .filter(lambda line: line[4].isdigit()) \
    .filter(lambda line: heplful_data_is_valid(line[6], line[7])) \
    .map(lambda line: (line[1], int(line[4]) * calculate_weight(float(line[6]), float(line[7])))) \
    .map(lambda line: (line[0], (line[1], 1))) \
    .reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1])) \
    .mapValues(lambda x: x[0] / x[1]) 

In [ ]:
result_data.coalesce(1).saveAsTextFile('/user-data/combined/results_with_helpful_data')